In [ ]:
# Authenticate to Earth Engine (run once)
import ee
ee.Authenticate()
ee.Initialize(project='openpas-463512')


In [ ]:
# Export annual precipitation to Google Drive (one file per asset)
import datetime
import ee

ee.Initialize(project='openpas-463512')

ASSET_IDS = [
    'projects/openpas-463512/assets/aracena',
    'projects/openpas-463512/assets/geres',
    'projects/openpas-463512/assets/ordesa',
    'projects/openpas-463512/assets/quercy',
    'projects/openpas-463512/assets/zorita',
]

DRIVE_FOLDER = 'GEE_OpenLandMap'  # change if you want another Drive folder
EXPORT_SCALE = 1000  # meters
# CRS must be explicit for Drive exports; use native EPSG:4326 for this dataset
CRS = 'EPSG:4326'
MAX_PIXELS = 1e13

MONTHLY_BANDS = ['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec']

dataset = ee.Image('OpenLandMap/CLM/CLM_PRECIPITATION_SM2RAIN_M/v01')
# Sum monthly bands and cast to Int16 (avoids Drive pixel type error)
annual_precip = dataset.select(MONTHLY_BANDS).reduce(ee.Reducer.sum()).rename('annual_precip_mm').toInt16()
timestamp = datetime.datetime.utcnow().strftime('%Y%m%dT%H%M%S')

tasks = []
for asset_id in ASSET_IDS:
    fc = ee.FeatureCollection(asset_id)
    region = fc.geometry()
    clipped = annual_precip.clip(region)
    safe_suffix = asset_id.split('/')[-1]
    task_name = f'openlandmap_precip_{safe_suffix}_{timestamp}'
    filename_prefix = f'{safe_suffix}_openlandmap_precip'
    task = ee.batch.Export.image.toDrive(
        image=clipped,
        description=task_name[:100],
        folder=DRIVE_FOLDER,
        fileNamePrefix=filename_prefix,
        region=region,
        scale=EXPORT_SCALE,
        crs=CRS,
        fileFormat='GeoTIFF',
        maxPixels=MAX_PIXELS,
    )
    task.start()
    tasks.append((task_name, asset_id))
    print(f"[TASK] Started Drive export for {asset_id} as {task_name}")

print('\nExports kicked off:', len(tasks))
print('Check task progress in https://code.earthengine.google.com/tasks')


In [ ]:
# OPTIONAL: Download small/medium areas directly to local disk (bypasses Drive)
# Note: getDownloadURL has size limits; large regions will fail. For big areas, use Drive exports above.
import pathlib
import requests
import ee

ee.Initialize(project='openpas-463512')

ASSET_IDS = [
    'projects/openpas-463512/assets/aracena',
    'projects/openpas-463512/assets/geres',
    'projects/openpas-463512/assets/ordesa',
    'projects/openpas-463512/assets/quercy',
    'projects/openpas-463512/assets/zorita',
]

EXPORT_SCALE = 1000
CRS = 'EPSG:4326'  # required for getDownloadURL
DOWNLOAD_DIR = pathlib.Path('downloads')
DOWNLOAD_DIR.mkdir(parents=True, exist_ok=True)

MONTHLY_BANDS = ['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec']
dataset = ee.Image('OpenLandMap/CLM/CLM_PRECIPITATION_SM2RAIN_M/v01')
annual_precip = dataset.select(MONTHLY_BANDS).reduce(ee.Reducer.sum()).rename('annual_precip_mm').toInt16()

for asset_id in ASSET_IDS:
    fc = ee.FeatureCollection(asset_id)
    region = fc.geometry()
    clipped = annual_precip.clip(region)
    safe_suffix = asset_id.split('/')[-1]
    url = clipped.getDownloadURL({
        'region': region,
        'scale': EXPORT_SCALE,
        'crs': CRS,
        'fileFormat': 'GeoTIFF',
    })
    out_path = DOWNLOAD_DIR / f"{safe_suffix}_openlandmap_precip.tif"
    print(f"[DOWNLOAD] {asset_id} -> {out_path}")
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(out_path, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
    print(f"[OK] Saved {out_path}")
